<a href="https://colab.research.google.com/github/LucasCapSilva/-JwtWithPermissionRole/blob/master/roboLinkedin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [377]:
from bs4 import BeautifulSoup as bs
import requests
import re

In [434]:
title = []
subtitle = []
location = []
dados = []
url= []
link = ''

In [287]:
link = 'https://www.linkedin.com/jobs/search?keywords=Java&location=Brazil&locationId=&geoId=106057199&f_TPR=&f_E=3&position=1&pageNum=0'
res = requests.get(link) 
soup = bs(res.text, 'lxml') 

In [288]:
file = soup.find_all('h3', class_="base-search-card__title")

In [289]:
for x in file:
  title.append(x.text.strip())

In [290]:
file = soup.find_all('a', class_="base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]")

In [291]:
for x in file:
  url.append(x['href'])

In [292]:
file = soup.find_all('h4', class_="base-search-card__subtitle")

In [293]:
for x in file:
  subtitle.append(x.text.strip())

In [294]:
file = soup.find_all('span', class_="job-search-card__location")

In [295]:
for x in file:
  location.append(x.text.strip())

In [376]:
dados = [];

In [297]:
for index, val in enumerate(title):
    res = requests.get(url[index]) 
    soup = bs(res.text, 'lxml')
    file = soup.find_all('p')
    for x in file:
        text = text + x.text.strip()
    dados.append({'title':title[index], 'content':text, 'subtitle':subtitle[index] , 'location':location[index] })

In [ ]:
dados[0]

In [442]:
def search_vacancies(num_page):
  text =""
  for x in range(num_page):
    title = []
    subtitle = []
    location = []
    url= []
    link = 'https://www.linkedin.com/jobs/search?keywords=Java&location=Brazil&locationId=&geoId=106057199&f_TPR=&f_E=3&position=1&pageNum='+str(x)
    res = requests.get(link) 
    soup = bs(res.text, 'lxml')
    file = soup.find_all('h3', class_="base-search-card__title") 
    for x in file:
        title.append(x.text.strip())
    file = soup.find_all('a', class_="base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]")
    for x in file:
        url.append(x['href'])
    file = soup.find_all('h4', class_="base-search-card__subtitle")
    for x in file:
        subtitle.append(x.text.strip())
    file = soup.find_all('span', class_="job-search-card__location")
    for x in file:
        location.append(x.text.strip())
    for index, val in enumerate(title):
      res = requests.get(url[index]) 
      soup = bs(res.text, 'lxml')
      file = soup.find_all('p')
      for x in file:
          text = text + x.text.strip()
      dados.append({'title':title[index], 'content':text, 'subtitle':subtitle[index] , 'location':location[index] })

In [443]:
search_vacancies(2)

In [444]:
dados[7]

{'title': 'DE&I - Software Developer, Finance Technology',
 'content': 'Entre para salvar DE&I Software Developer Engineer I, Intech na Amazon.As indicações dobram suas chances de conseguir uma entrevista na AmazonReceba alertas de novas vagas de Engenheiro de software em: São Paulo, SP.Receber atualizações por e-mail sobre novas vagas de Engenheiro de software em: São Paulo, SPAo criar este alerta de vaga, você aceita o Contrato do Usuário e a Política de Privacidade do LinkedIn. Você pode cancelar a inscrição para receber atualizações por e-mail a qualquer hora.Entre para criar maisSeu alerta de vaga foi definidoClique no link que enviamos por e-mail para  para confirmar seu e-mail e ativar seu alerta de vaga.Entre para criar seu alerta de vaga de Engenheiro de software em: São Paulo, SP.Entre para salvar DE&I Developer I Intech na Amazon.As indicações dobram suas chances de conseguir uma entrevista na AmazonReceba alertas de novas vagas de Engenheiro de software em: São Paulo, SP.Re

In [ ]:
for index, val in enumerate(title):
      res = requests.get(url[index]) 
      soup = bs(res.text, 'lxml')
      file = soup.find_all('p')
      for x in file:
          print(x.text.strip())